In [ ]:
%cd ..

In [ ]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob
import re

In [ ]:
def extract_qa(content) -> list[str]:
	# Regex to match <qa>...</qa> and <qa>..."answer"...
	qa_pattern = re.compile(r'<qa>(.*?)</qa>|<qa>(.*?"answer".*?"option \d:.*?)<|eot_id|>', re.DOTALL)
	
	matches = qa_pattern.findall(content)
	
	# Flatten the list of tuples and filter out empty strings
	matches = [match[0] if match[0] else match[1] for match in matches]
	
	return matches

files = glob("data/generated_qa/*/*/*.txt")

In [ ]:
n_error = 0
ignore_match = '"question": "Question text goes here"'

In [ ]:
def get_questions(file : str):
	raw = open(file, "r").read()
	matches = extract_qa(raw)
	matches = [i for i in matches if i.strip()]
	# if len(matches) != 6:
	# 	print(len(matches), file)
	for match in matches:
		if not len(match.strip()) or ignore_match in match:
			continue
		match = match.strip()
		if match.endswith(","):
			match = match[:-1]
		match = "{" + match + "}"
		try:
			match = json.loads(match.replace('"answer": "answer":', '"answer":').replace('""', '”"'))
			match["source"] = file
			yield match
		except:
			pass

In [ ]:
questions = [
	list(get_questions(file)) for file in files
]

In [ ]:
len(questions)

In [ ]:
questions[10]

In [ ]:
total_questions = sum(questions, start=[])

len(total_questions)

In [ ]:
df = pd.DataFrame(total_questions)
df.head()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df[df["option 5"].isna()]["source"].values[:10]

In [ ]:
options = df["answer"].apply(lambda x: str(x).split(" ")[1][0])

In [ ]:
options.value_counts()

In [ ]:
df[options == "o"]

In [ ]:
df[options == "m"]

In [ ]:
df[options == "a"]

In [ ]:
df[options == "p"]

In [ ]:
df_filtered = df[options.str.isdigit()].copy()
print(df_filtered.shape)

In [ ]:
len(options)

In [ ]:
options = df_filtered["answer"].apply(lambda x: int(str(x).split(" ")[1][0]))
df_filtered["answer_index"] = options.values

df_filtered.sample(5)

In [ ]:
df_filtered.columns

In [ ]:
df_filtered[[
    'question', 'option 1', 'option 2', 'option 3', 'option 4', 'option 5', 'answer_index'
]].to_csv("data/full_data/generated_v1.csv", index=False)

In [ ]:
teleqna = pd.DataFrame(json.load(open("data/TeleQnA.json", "r"))).T
test_qna = pd.DataFrame(json.load(open("data/zindi_data/TeleQnA_testing1.json",))).T


teleqna.shape, test_qna.shape

In [ ]:
teleqna.sample(5)

In [ ]:
teleqna["answer_index"] = teleqna["answer"].apply(lambda x: int(str(x).split()[1][0]))
teleqna["answer_index"].value_counts()

In [ ]:
teleqna.columns

In [ ]:
"]".__contains__("]")

In [ ]:
teleqna["question"].apply(lambda x: str(x).split("[")[-1].__contains__("]")).sum(), len(teleqna)

In [ ]:
teleqna[teleqna.index.isin(test_qna.index)][
    ['question', 'option 1', 'option 2', 'option 3', 'option 4', 'option 5', 'answer_index']
].to_csv("data/full_data/test_data.csv", index=False)

teleqna[~teleqna.index.isin(test_qna.index)][
    ['question', 'option 1', 'option 2', 'option 3', 'option 4', 'option 5', 'answer_index']
].to_csv("data/full_data/train_data.csv", index=False)